In [1]:
#!pip install beautifulsoup4
#!pip install lxml
#!pip install splinter
#!pip install requests
#!pip install rethinkdb
#!pip install pymongo

In [8]:
import requests as r
import time
import rethinkdb as redb
import pymongo

from bs4 import BeautifulSoup
from splinter import Browser
from time import sleep
from pymongo import MongoClient

browser = Browser('chrome', executable_path='./chromedriver.exe')
browser.visit("http://www.pacific.com.tw/Object/ObjectList")
sleep(3)

#型態選公寓，華廈，住宅大樓，透天，套房
browser.find_by_css(".cam_select")[4].click()
sleep(1)
browser.find_by_xpath("/html/body/div[1]/section[1]/div/div[7]/div/div[1]/label")[0].click()
browser.find_by_xpath("/html/body/div[1]/section[1]/div/div[7]/div/div[2]/label")[0].click()
browser.find_by_xpath("/html/body/div[1]/section[1]/div/div[7]/div/div[3]/label")[0].click()
browser.find_by_xpath("/html/body/div[1]/section[1]/div/div[7]/div/div[4]/label")[0].click()
browser.find_by_xpath("/html/body/div[1]/section[1]/div/div[7]/div/div[5]/label")[0].click()
sleep(1)
browser.find_by_xpath("/html/body/div[1]/section[1]/div/div[2]/div/div[8]/a")[0].click()
sleep(3)

#爬取要的資料
def wait():
    while True:
        is_loaded = browser.evaluate_script("document.readyState")
        if is_loaded == 'complete':
            break
        else:
            sleep(1)

num = browser.find_by_xpath('/html/body/div[1]/section[2]/div[2]/div[2]/i')
turnover = int(num.text)//10  #總資料比數除以10得出全爬應該要爬的頁數

page = turnover #一次要爬幾頁

data_lists = []
for j in range(0,page):

    links = []
    for i in range(0, 10):
        links.append(browser.find_by_css(".house_list")[i]['href'])

    for i in range(1,11):
        browser.find_by_xpath("/html/body/div[1]/section[2]/ul/li[%i]/a/div[2]/h1" % (i)).click()
        wait()
        sleep(3)
        browser.windows.current = browser.windows[1]
        data_dict = {}

        name = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/h1")
        address = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/h2")
        price_ten_thousand = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/div[2]/div/b")
        area = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/ul/li[1]")
        structure = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/ul/li[3]")
        typeOfBuilding = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/ul/li[5]")
        height = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/ul/li[6]")
        purpose = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/ul/li[7]")
        age = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[1]/td[1]")
        side = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[1]/td[2]")
        fee = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[2]/td[1]")
        sunLight = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[2]/td[2]")
        positionTo = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[3]/td[1]")
        nearbyMarket = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[3]/td[2]")
        nearbyMRT = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[5]/td[1]")
        nearbyBus = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tbody/tr[5]/td[2]")
        features = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[2]/div/table/tfoot/tr/td")
        mainHouseArea = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[1]/td[1]")
        subHouseArea = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[1]/td[2]")
        publicArea = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[2]/td[1]")
        landOwn = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[2]/td[2]")
        publicParkin = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[3]/td[1]")
        privateParking = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[3]/td[2]")
        usageSeparation = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[3]/div/table/tbody/tr[4]/td")
        unit_price = browser.find_by_xpath("/html/body/div[1]/div[1]/section[2]/ul/li[1]/div/div[2]/div[1]/ul/li[2]")
        try:
            pic = browser.find_by_xpath('//*[@id="ObjPic1"]/div/img')[0]['src']
        except:
            pic = ''

        #公有的欄位
        try:
            num = links[(i - 1)].split('saleID=')[1]
            data_dict['_id'] = 'pac' + num
        except:
            data_dict['_id'] = ''
        data_dict['web'] = 'pac'

        try:
            city = address.text.split('市')[0]
            sec = address.text.split('市')[1].split('區')[0]
            road = address.text.split('市')[1].split('區')[1]
            data_dict['road'] = road
            data_dict['city'] = city + '市'
            data_dict['sec'] = sec + '區'
        except:
            data_dict['road'] = ''
            data_dict['city'] = ''
            data_dict['sec'] = ''

        data_dict['house_link'] = links[(i - 1)]
        data_dict['crawl_time'] = time.strftime("%Y/%m/%d %H:%M:%S")

        try:
            data_dict['type'] = typeOfBuilding.text.split('：')[1]
        except:
            data_dict['type'] = ''

        try:
            data_dict['car_size'] = publicParkin.text.split('坪')[0]
        except:
            data_dict['car_size']

        try:
            data_dict['lsize'] = float(landOwn.text.split('坪')[0])
        except:
            data_dict['lsize'] = ''

        try:
            data_dict['hsize'] = float(area.text.split('：')[1].split('坪')[0])
        except:
            data_dict['hsize'] = ''

        try:
            data_dict['msize'] = float(mainHouseArea.text.split('坪')[0])
        except:    
            data_dict['msize'] = ''

        try:
            data_dict['psize'] = float(publicArea.text.split('坪')[0])
        except:
            data_dict['psize'] = ''

        try:    
            data_dict['osize_o'] = float(subHouseArea.text.split('坪')[0])
        except:
            data_dict['osize_o'] = ''
        try:
            data_dict['year'] = float(age.text.split('年')[0])
        except:
            data_dict['year'] = ''

        try:
            height1 = height.text.split('：')[1]
            if height1 != "":
                floor = height1.split('/')[0].split('樓')[0]
            else:
                floor = ""
            if height1 != "":
                total_floor = height1.split('/')[1].split('樓')[0]
            else:
                total_floor = ""
            data_dict['total_floor'] = float(total_floor)
            data_dict['floor'] = float(floor)
        except:
            data_dict['total_floor'] = ''
            data_dict['floor'] = ''

        try:
            x = structure.text.split('：')[1]
            xx = "-".join(x)
            y= xx.split("-")
            if '廳' in y:
                rest_num =  y.index("廳")
                rest_num = y[rest_num-1]
            else:
                rest_num = ""
            if '房' in y:
                room_num = y.index("房")
                room_num = y[room_num-1]
            else:
                room_num = ""
            if '衛' in y:
                wc_num = y.index("衛")
                wc_num = y[wc_num-1]
            else:
                wc_num = ""
            data_dict['rest_num'] = float(rest_num)
            data_dict['room_num'] = float(room_num)
            data_dict['wc_num'] = float(wc_num)
        except:
            data_dict['rest_num'] = ''
            data_dict['room_num'] = ''
            data_dict['wc_num'] = ''
        try:
            data_dict['total_price'] = float(price_ten_thousand.text)
        except:
            data_dict['total_price'] = ''

        data_dict['pic'] = pic

        try:
            data_dict['intro'] = features.text
        except:
            data_dict['intro'] = ''

        #其他特殊欄位
        try:
            data_dict['name'] = name.text
        except:
            data_dict['name'] = ''
             
        try:
            data_dict['purpose'] = purpose.text.split('：')[1]
        except:
            data_dict['purpose'] = ''
            
        try:
            data_dict['side'] = side.text
        except:
            data_dict['side'] = ''
            
        try:
            data_dict['fee'] = fee.text
        except:
            data_dict['fee'] = ''
            
        try:
            data_dict['sunLight'] = sunLight.text
        except:
            data_dict['sunLight'] = ''
        
        
        try:
            data_dict['positionTo'] = positionTo.text
        except:
            data_dict['positionTo'] = ''
            
        try:
            data_dict['nearbyMarket'] = nearbyMarket.text
        except:
            data_dict['nearbyMarket'] = ''
        
        try:
            data_dict['nearbyMRT'] = nearbyMRT.text
        except:
            data_dict['nearbyMRT'] = ''
        
        try:
            data_dict['nearbyBus'] = nearbyBus.text
        except:
            data_dict['nearbyBus'] = ''
        
        try:
            data_dict['privateParking'] = privateParking.text
        except:
            data_dict['privateParking'] = ''
        
        try:
            data_dict['usageSeparation'] = usageSeparation.text
        except:
            data_dict['usageSeparation'] = ''

        browser.windows.current = browser.windows[0]
        browser.windows[0].close_others()  

        data_lists.append(data_dict)

    browser.find_by_xpath("/html/body/div[1]/section[2]/div[3]/a[2]/i").click()
    sleep(3)
    wait()

data_lists
    
#rethinkdb裡database是 pacific_realtor，table有分 data_Taipei 跟 data_NewTaipei
#conn = redb.connect("localhost", 28015).repl()
#redb.db('pacific_realtor').table('data_Taipei').insert(data_lists).run(conn)

[{'_id': 'pac071971',
  'car_size': '',
  'city': '臺北市',
  'crawl_time': '2017/05/28 23:43:08',
  'fee': '590元/ 月繳',
  'floor': 11.0,
  'house_link': 'http://www.pacific.com.tw/Object/ObjectDetail/?saleID=071971',
  'hsize': 7.32,
  'intro': '台北車站旁交通便利\n,位處市中心生活機能佳\n,可設藉公司登記，超低總價投資首選',
  'lsize': 0.84,
  'msize': 4.99,
  'name': '延平超值套房071971',
  'nearbyBus': '延平鄭州路口',
  'nearbyMRT': '北門',
  'nearbyMarket': '傳統市場',
  'osize_o': '',
  'pic': 'http://img.pacific.com.tw/bimg/071971A.jpg',
  'positionTo': '東',
  'privateParking': '',
  'psize': 2.32,
  'purpose': '商業用',
  'rest_num': '',
  'road': '延平北路一段',
  'room_num': '',
  'sec': '大同區',
  'side': '否',
  'sunLight': '',
  'total_floor': 11.0,
  'total_price': 388.0,
  'type': '住宅大樓',
  'usageSeparation': '第參種商業區',
  'wc_num': '',
  'web': 'pac',
  'year': 36.0},
 {'_id': 'pac064785',
  'car_size': '',
  'city': '臺北市',
  'crawl_time': '2017/05/28 23:43:14',
  'fee': '350元/ 月繳',
  'floor': 3.0,
  'house_link': 'http://www.pacific.com.tw/O